In [1]:
# imports
import numpy as np
import pandas as pd
import pickle
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from tqdm.auto import tqdm
from multiprocessing import Pool

In [2]:
# connect to local elastic search host
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST])
INDEX = 'info_clone'
NEWINDEX = 'info_clone2'
es.ping()

True

In [3]:
t = es.indices.stats(index=INDEX)
SIZE = t['_all']['primaries']['docs']['count']
print(SIZE)

In [4]:
s = Search(using=es, index=NEWINDEX)
q = Q("wildcard", name='*')
query = s.query(q) #.extra(size=SIZE)
#response = query.execute()
#print(len(response))

In [29]:
# Find files that contain no markdown or code
def check_empty(es, nid):
    mkdn = es.get(index='markdown', id=nid, ignore=[404])
    if '_source' in mkdn:
        mkdnlist = mkdn['_source']['markdown']
    else:
        mkdnlist = None
    code = es.get(index='code', id=nid, ignore=[404])
    if '_source' in code:
        codelist = code['_source']['code']
    else:
        codelist = None
    comments = es.get(index='comments2', id=nid, ignore=[404])
    if '_source' in comments:    
        cmtlist = comments['_source']['comments']
    else:
        cmtlist = None
    if mkdnlist == [] and codelist == [] and cmtlist == []:
        return True
    else:
        return False

In [30]:
empty_files = []
for hit in s.scan():
    nid = hit.meta.id
    print(nid, end='\r')
    if check_empty(es, nid):
        #print(nid, end='\r')
        empty_files.append(nid)

In [31]:
len(empty_files)
# 1749 before no '_source' + 91

91

In [32]:
with open('empty2.pkl', 'wb') as f:
    pickle.dump(empty_files, f)

In [33]:
with open('empty2.pkl', 'rb') as f:
     empty_files2 = pickle.load(f)

In [ ]:
#es.delete(index=NEWINDEX, id=nid)
#mdelete

In [ ]:
# Does not run properly
SLICES = 2

def test(slice_no):
    test = []
    for i in range(10):
        test.append(i)
    return test

pool = Pool(SLICES)                                                             
all_test = pool.map(test, range(SLICES))
all_test = [i for l in all_test for i in l]

# POOL NOT WORKING
# SLICES = 5                                                                      

# def dump_slice(slice_no):
#     empty_files = []
#     s = Search()                                                                
#     s = s.extra(slice={"id": slice_no, "max": SLICES})                          
#     for hit in s.scan():                                                          
#         nid = hit.meta.id
#         if check_empty(es, nid, name):
#             empty_files.append(nid)
#     return empty_files

# pool = Pool(SLICES)                                                             
# pool.map(dump_slice, range(SLICES))

In [34]:
#POST /NEWINDEX/_delete_by_query
idsbody = {
    "query": {
        "ids" : {
        "values" : empty_files2
        }
    }    
}

es.delete_by_query(index=NEWINDEX, body=idsbody)

{'batches': 1,
 'deleted': 91,
 'failures': [],
 'noops': 0,
 'requests_per_second': -1.0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'throttled_until_millis': 0,
 'timed_out': False,
 'took': 1215,
 'total': 91,
 'version_conflicts': 0}

In [13]:
response = query.execute()

In [17]:
response[0].to_dict()

{'html_url': 'https://github.com/chao1224/Steiner-Tree-in-Flight-Scheduling/blob/2641a6bdcda7b1941c04da3c707882c647dd5ebb/project_report.ipynb',
 'language': 'julia',
 'name': 'project_report.ipynb'}

In [21]:
x = es.get(index=NEWINDEX, id=1250687)

In [23]:
x['_source']

{'html_url': 'https://github.com/toastmaker/UCD-Python/blob/e626a9ec77962c6386c1f4c9ebd63ddbcf3e609b/UCD-ch6-speed.ipynb',
 'language': 'unknown',
 'name': 'UCD-ch6-speed.ipynb'}

In [35]:
print(1749 + 91)

1840


In [36]:
print(980937-1840)

979097
